<a href="https://colab.research.google.com/github/carlos-alves-one/-Crypto-Electronic-Medical-Records/blob/main/crypto_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Cryptography Libraries

In [ ]:
# Import the 'hashes' module from the cryptography library's hazmat primitives, used for cryptographic hashing
from cryptography.hazmat.primitives import hashes

# Import 'Encoding', 'PublicFormat', and 'load_der_public_key' from the serialization module for encoding formats,
# public key formats, and loading DER-encoded public keys, respectively
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, load_der_public_key

# Import the 'ec' (Elliptic Curve cryptography) and 'utils' modules from the asymmetric part of the hazmat primitives
# for performing operations with elliptic curves and various utility functions
from cryptography.hazmat.primitives.asymmetric import ec, utils


In [1]:
class EMR:
    def __init__(self, Dr_hash, Patient_hash, Dr_public_key, prescription, nonce, signature, txid):
        self.Dr_hash = Dr_hash
        self.Patient_hash = Patient_hash
        self.Dr_public_key = Dr_public_key
        self.prescription = prescription
        self.nonce = nonce
        self.signature = signature
        self.txid = txid

    def verify(self, Dr_previous_nonce):
        # Check the length of hashes
        if len(self.Dr_hash) != 20 or len(self.Patient_hash) != 20:
            raise Exception("Hash is wrong length")

        # Check the hash of the doctor's public key
        if calculate_sha1_hash(self.Dr_public_key) != self.Dr_hash:
            raise Exception("Invalid doctor public key")

        # Check the length of the prescription
        if not isinstance(self.prescription, str) or len(self.prescription.encode('utf-8')) > 200:
            raise Exception("Invalid prescription")

        # Check the nonce
        if self.nonce != Dr_previous_nonce + 1:
            raise Exception("Invalid nonce")

        # Check the txid
        expected_txid = calculate_txid(self.Dr_hash, self.Patient_hash, self.Dr_public_key, self.prescription, self.nonce, self.signature)
        if self.txid != expected_txid:
            raise Exception("Invalid txid")

        # Verify the signature
        signature_hash = calculate_signature_hash(self.Patient_hash, self.prescription, self.nonce)
        key = load_der_public_key(self.Dr_public_key)
        try:
            key.verify(self.signature, signature_hash, ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        except:
            raise Exception("Invalid signature")

def calculate_sha1_hash(data):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(data)
    return digest.finalize()

def calculate_txid(Dr_hash, Patient_hash, Dr_public_key, prescription, nonce, signature):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(Dr_hash)
    digest.update(Patient_hash)
    digest.update(Dr_public_key)
    digest.update(prescription.encode('utf-8'))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    digest.update(signature)
    return digest.finalize()

def calculate_signature_hash(Patient_hash, prescription, nonce):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(Patient_hash)
    digest.update(prescription.encode('utf-8'))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    return digest.finalize()
